# Test3-Other Model Croass Validation and Testing Set : Dataset-tmf

In [2]:
from brbmdl import dataset_loading, process_dl_dataset
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import Adam

In [3]:
features = ['macd', 'rsi', 'k', 'd', 'r', 'close',
            'before1day', 'before2day', 'before3day', 'before4day','before5day']
target = 'dailyreturn'

In [4]:
X_train, y_train = dataset_loading(features, target, 
                                   './data/tsmc_train_data.csv',
                                   './data/foxconn_train_data.csv',
                                   './data/mediatek_train_data.csv'
                                   )
# X_test, y_test = dataset_loading(features, target, 
#                                  './data/tsmc_test_data.csv',
#                                  './data/foxconn_test_data.csv',
#                                  './data/mediatek_test_data.csv')

In [6]:
stocks = ['tsmc', 'mediatek', 'foxconn']

In [7]:
# window_size = 5
# convertor = LabelEncoder()
# y_train = convertor.fit_transform(process_dataset(y_train, target))
# y_test = convertor.fit_transform(process_dataset(y_test, target))
# X_train, y_train = np.array(X_train), np.array(y_train)
# X_test, y_test = np.array(X_test), np.array(y_test)
# X_train, y_train = create_multivariate_dataset(X_train, y_train, window_size)
# X_test, y_test = create_multivariate_dataset(X_test, y_test, window_size)

#### LSTM

In [9]:
val_df = pd.concat([X_train, y_train], axis=1)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

accuracy_list, precision_list, recall_list, f1_list = [], [], [], []

for train_idx, val_idx in kf.split(val_df):
    X_train_part = X_train.iloc[train_idx]
    y_train_label_part = y_train.iloc[train_idx]
    X_val_part = X_train.iloc[val_idx]
    y_val_part = y_train.iloc[val_idx]
    X_train_part, X_val_part, y_train_label_part, y_val_part = process_dl_dataset(X_train_part, X_val_part, y_train_label_part, y_val_part, target)

    model_lstm = Sequential()
    model_lstm.add(LSTM(64, input_shape=(X_train_part.shape[1], X_train_part.shape[2]), return_sequences=False))
    model_lstm.add(Dense(64, activation='sigmoid'))
    model_lstm.add(Dense(1, activation='sigmoid'))  # 二元分類使用 sigmoid 激活函數
    model_lstm.compile(optimizer=Adam(learning_rate=9.65E-03), loss='binary_crossentropy', metrics=['accuracy'])
    model_lstm.fit(X_train_part, y_train_label_part, epochs=10, batch_size=32, validation_data=(X_val_part, y_val_part))
    # loss, accuracy = model_lstm.evaluate(X_val_part, y_val_part)

    y_val_pred = model_lstm.predict(X_val_part)
    y_val_pred = (y_val_pred > 0.5).astype(int)  # 將概率轉換為 0 或 1
    accuracy = accuracy_score(y_val_part, y_val_pred)
    precision = precision_score(y_val_part, y_val_pred, average='macro')
    recall = recall_score(y_val_part, y_val_pred, average='macro')
    f1 = f1_score(y_val_part, y_val_pred, average='macro')
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)

print(f"LSTM 驗證集 5 折平均 Accuracy: {np.mean(accuracy_list) * 100:.2f}%")
print(f"LSTM 驗證集 5 折平均 Precision: {np.mean(precision_list) * 100:.2f}%")
print(f"LSTM 驗證集 5 折平均 Recall: {np.mean(recall_list) * 100:.2f}%")
print(f"LSTM 驗證集 5 折平均 F1 Score: {np.mean(f1_list) * 100:.2f}%")

Epoch 1/10


/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5722 - loss: 0.6617 - val_accuracy: 0.6377 - val_loss: 0.6484
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6886 - loss: 0.6037 - val_accuracy: 0.6407 - val_loss: 0.6112
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6999 - loss: 0.5845 - val_accuracy: 0.6387 - val_loss: 0.6128
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6841 - loss: 0.5872 - val_accuracy: 0.6426 - val_loss: 0.6457
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6744 - loss: 0.5952 - val_accuracy: 0.6565 - val_loss: 0.6225
Epoch 6/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6711 - loss: 0.5972 - val_accuracy: 0.6417 - val_loss: 0.6244
Epoch 7/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6932 - loss: 0.6032 - val_accuracy: 0.6426 - val_loss: 0.6728
Epoch 8/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6870 - loss: 0.5924 - val_accuracy: 0.6436 - val_

/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6106 - loss: 0.6476 - val_accuracy: 0.6989 - val_loss: 0.5852
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6700 - loss: 0.6086 - val_accuracy: 0.6881 - val_loss: 0.5850
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6850 - loss: 0.5873 - val_accuracy: 0.6960 - val_loss: 0.5732
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6735 - loss: 0.6087 - val_accuracy: 0.6950 - val_loss: 0.5766
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6680 - loss: 0.6128 - val_accuracy: 0.6930 - val_loss: 0.5679
Epoch 6/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6695 - loss: 0.5970 - val_accuracy: 0.6890 - val_loss: 0.5538
Epoch 7/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6864 - loss: 0.5812 - val_accuracy: 0.6762 - val_loss: 0.5745
Epoch 8/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6704 - loss: 0.6003 - val_accuracy: 0.6841 - val_

...

127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5910 - loss: 0.6643 - val_accuracy: 0.6328 - val_loss: 0.6650
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6864 - loss: 0.6050 - val_accuracy: 0.6673 - val_loss: 0.5996
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6789 - loss: 0.5969 - val_accuracy: 0.6654 - val_loss: 0.5896
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6813 - loss: 0.5992 - val_accuracy: 0.6584 - val_loss: 0.5998
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6828 - loss: 0.5908 - val_accuracy: 0.6742 - val_loss: 0.5997
Epoch 6/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6828 - loss: 0.5800 - val_accuracy: 0.6732 - val_loss: 0.5804
Epoch 7/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6629 - loss: 0.5867 - val_accuracy: 0.6644 - val_loss: 0.5876
Epoch 8/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6754 - loss: 0.5897 - val_accuracy: 0.6713 - val_

/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5564 - loss: 0.7019 - val_accuracy: 0.6841 - val_loss: 0.5858
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6827 - loss: 0.5951 - val_accuracy: 0.7058 - val_loss: 0.5668
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6645 - loss: 0.6076 - val_accuracy: 0.6920 - val_loss: 0.5712
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6666 - loss: 0.6029 - val_accuracy: 0.7058 - val_loss: 0.5573
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6679 - loss: 0.5977 - val_accuracy: 0.6713 - val_loss: 0.5895
Epoch 6/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6768 - loss: 0.5843 - val_accuracy: 0.6723 - val_loss: 0.5700
Epoch 7/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6892 - loss: 0.5775 - val_accuracy: 0.7009 - val_loss: 0.5916
Epoch 8/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6810 - loss: 0.5820 - val_accuracy: 0.7029 - val_

/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5878 - loss: 0.6605 - val_accuracy: 0.6900 - val_loss: 0.5893
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6576 - loss: 0.6087 - val_accuracy: 0.6713 - val_loss: 0.5956
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6744 - loss: 0.5944 - val_accuracy: 0.6969 - val_loss: 0.5694
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6750 - loss: 0.5950 - val_accuracy: 0.6871 - val_loss: 0.5755
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6798 - loss: 0.5930 - val_accuracy: 0.6565 - val_loss: 0.5941
Epoch 6/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6771 - loss: 0.5972 - val_accuracy: 0.6693 - val_loss: 0.5932
Epoch 7/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6579 - loss: 0.6035 - val_accuracy: 0.6979 - val_loss: 0.5765
Epoch 8/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6833 - loss: 0.5744 - val_accuracy: 0.6821 - val_

In [10]:
model = 'LSTM'
N = 1
for stock in stocks:
    lstm_accs, lstm_pres, lstm_recs, lstm_f1s = [], [], [], []

    X_test, y_test = dataset_loading(features, target, f'./{stock}_test_data.csv',)
    X_train_dl, X_test_dl, y_train_dl, y_test_dl = process_dl_dataset(X_train, X_test, y_train, y_test, target)

    for i in range(N):
        model_lstm = Sequential()
        model_lstm.add(LSTM(64, input_shape=(X_train_dl.shape[1], X_train_dl.shape[2]), return_sequences=False))
        model_lstm.add(Dense(64, activation='sigmoid'))
        model_lstm.add(Dense(1, activation='sigmoid'))  # 二元分類使用 sigmoid 激活函數
        model_lstm.compile(optimizer=Adam(learning_rate=9.65E-03), loss='binary_crossentropy', metrics=['accuracy'])
        model_lstm.fit(X_train_dl, y_train_dl, epochs=10, batch_size=32, validation_data=(X_test_dl, y_test_dl))
        # loss, accuracy = model_lstm.evaluate(X_test_dl, y_test_dl)

        y_pred = model_lstm.predict(X_test_dl)
        y_pred = (y_pred > 0.5).astype(int)  # 將概率轉換為 0 或 1

        accuracy = accuracy_score(y_test_dl, y_pred)
        precision = precision_score(y_test_dl, y_pred, average='macro')
        recall = recall_score(y_test_dl, y_pred, average='macro')
        f1 = f1_score(y_test_dl, y_pred, average='macro')
        lstm_accs.append(accuracy)
        lstm_pres.append(precision)
        lstm_recs.append(recall)
        lstm_f1s.append(f1)

    print(f'{stock} LSTM Accuracy: {np.mean(lstm_accs) * 100:.2f}%')
    print(f'{stock} LSTM Precision: {np.mean(lstm_pres) * 100:.2f}%')
    print(f'{stock} LSTM Recall: {np.mean(lstm_recs) * 100:.2f}%')
    print(f'{stock} LSTM F1 Score: {np.mean(lstm_f1s) * 100:.2f}%')

Epoch 1/10


/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6185 - loss: 0.6395 - val_accuracy: 0.6989 - val_loss: 0.5988
Epoch 2/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6800 - loss: 0.5966 - val_accuracy: 0.6906 - val_loss: 0.5829
Epoch 3/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6907 - loss: 0.5899 - val_accuracy: 0.6851 - val_loss: 0.5952
Epoch 4/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6740 - loss: 0.6048 - val_accuracy: 0.6948 - val_loss: 0.6309
Epoch 5/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6843 - loss: 0.5966 - val_accuracy: 0.6934 - val_loss: 0.6111
Epoch 6/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6793 - loss: 0.5946 - val_accuracy: 0.6934 - val_loss: 0.5890
Epoch 7/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6672 - loss: 0.5934 - val_accuracy: 0.6961 - val_loss: 0.5827
Epoch 8/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6737 - loss: 0.5827 - val_accuracy: 0.7017 - val_

/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6450 - loss: 0.6342 - val_accuracy: 0.6588 - val_loss: 0.6036
Epoch 2/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6771 - loss: 0.6113 - val_accuracy: 0.6727 - val_loss: 0.6184
Epoch 3/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6710 - loss: 0.6013 - val_accuracy: 0.6699 - val_loss: 0.6068
Epoch 4/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6879 - loss: 0.5929 - val_accuracy: 0.6602 - val_loss: 0.6258
Epoch 5/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6650 - loss: 0.6055 - val_accuracy: 0.6671 - val_loss: 0.5947
Epoch 6/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6684 - loss: 0.6004 - val_accuracy: 0.6768 - val_loss: 0.6003
Epoch 7/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6818 - loss: 0.5814 - val_accuracy: 0.6575 - val_loss: 0.6186
Epoch 8/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6584 - loss: 0.6062 - val_accuracy: 0.6575 - val_

/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5726 - loss: 0.6978 - val_accuracy: 0.6823 - val_loss: 0.6083
Epoch 2/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6683 - loss: 0.6157 - val_accuracy: 0.6906 - val_loss: 0.6123
Epoch 3/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6761 - loss: 0.5989 - val_accuracy: 0.6630 - val_loss: 0.5881
Epoch 4/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6768 - loss: 0.5981 - val_accuracy: 0.6865 - val_loss: 0.5840
Epoch 5/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6906 - loss: 0.5848 - val_accuracy: 0.6713 - val_loss: 0.5934
Epoch 6/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6787 - loss: 0.5853 - val_accuracy: 0.6823 - val_loss: 0.6153
Epoch 7/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6773 - loss: 0.5860 - val_accuracy: 0.6754 - val_loss: 0.5922
Epoch 8/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6729 - loss: 0.5780 - val_accuracy: 0.6989 - val_

#### CNN

In [18]:
val_df = pd.concat([X_train, y_train], axis=1)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

accuracy_list, precision_list, recall_list, f1_list = [], [], [], []

for train_idx, val_idx in kf.split(val_df):
    X_train_part = X_train.iloc[train_idx]
    y_train_label_part = y_train.iloc[train_idx]
    X_val_part = X_train.iloc[val_idx]
    y_val_part = y_train.iloc[val_idx]
    X_train_part, X_val_part, y_train_label_part, y_val_part = process_dl_dataset(X_train_part, X_val_part, y_train_label_part, y_val_part, target)

    model_cnn = Sequential()
    model_cnn.add(Conv1D(64, 3, activation='sigmoid', input_shape=(X_train_part.shape[1], X_train_part.shape[2]))) # 添加 Conv1D 層，對時間序列進行卷積操作
    model_cnn.add(MaxPooling1D(2))
    model_cnn.add(Flatten())
    # model_cnn.add(Dense(64, activation='sigmoid'))
    model_cnn.add(Dense(1, activation='sigmoid'))  # 二元分類使用 sigmoid 激活函數
    model_cnn.compile(optimizer=Adam(learning_rate=9.65E-03), loss='binary_crossentropy', metrics=['accuracy'])
    model_cnn.fit(X_train_part, y_train_label_part, epochs=10, batch_size=32, validation_data=(X_val_part, y_val_part))
    # loss, accuracy = model_cnn.evaluate(X_val_part, y_val_part)

    y_val_pred = model_cnn.predict(X_val_part)

    y_val_pred = (y_val_pred > 0.5).astype(int)  # 將概率轉換為 0 或 1
    accuracy = accuracy_score(y_val_part, y_val_pred)
    precision = precision_score(y_val_part, y_val_pred, average='macro')
    recall = recall_score(y_val_part, y_val_pred, average='macro')
    f1 = f1_score(y_val_part, y_val_pred, average='macro')
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)

print(f"CNN 驗證集 5 折平均 Accuracy: {np.mean(accuracy_list) * 100:.2f}%")
print(f"CNN 驗證集 5 折平均 Precision: {np.mean(precision_list) * 100:.2f}%")
print(f"CNN 驗證集 5 折平均 Recall: {np.mean(recall_list) * 100:.2f}%")
print(f"CNN 驗證集 5 折平均 F1 Score: {np.mean(f1_list) * 100:.2f}%")

Epoch 1/10


/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5701 - loss: 0.6755 - val_accuracy: 0.6683 - val_loss: 0.6106
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7070 - loss: 0.5717 - val_accuracy: 0.6950 - val_loss: 0.5906
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7178 - loss: 0.5454 - val_accuracy: 0.7068 - val_loss: 0.5410
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1000us/step - accuracy: 0.7491 - loss: 0.5055 - val_accuracy: 0.7660 - val_loss: 0.4789
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7921 - loss: 0.4494 - val_accuracy: 0.8055 - val_loss: 0.4308
Epoch 6/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 952us/step - accuracy: 0.8232 - loss: 0.4053 - val_accuracy: 0.8085 - val_loss: 0.3929
Epoch 7/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 940us/step - accuracy: 0.8425 - loss: 0.3662 - val_accuracy: 0.8519 - val_loss: 0.3716
Epoch 8/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 938us/step - accuracy: 0.8594 - loss: 0.3404 - val_accuracy: 0.82

/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5858 - loss: 0.6756 - val_accuracy: 0.7137 - val_loss: 0.5762
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 956us/step - accuracy: 0.7045 - loss: 0.5697 - val_accuracy: 0.7374 - val_loss: 0.5199
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7385 - loss: 0.5304 - val_accuracy: 0.7917 - val_loss: 0.4596
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7693 - loss: 0.4793 - val_accuracy: 0.8411 - val_loss: 0.4060
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 966us/step - accuracy: 0.8208 - loss: 0.4200 - val_accuracy: 0.8628 - val_loss: 0.3595
Epoch 6/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 975us/step - accuracy: 0.8363 - loss: 0.3821 - val_accuracy: 0.8707 - val_loss: 0.3266
Epoch 7/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 945us/step - accuracy: 0.8516 - loss: 0.3582 - val_accuracy: 0.8500 - val_loss: 0.3363
Epoch 8/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 904us/step - accuracy: 0.8523 - loss: 0.3482 - val_accuracy: 0.8

...

127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5755 - loss: 0.7033 - val_accuracy: 0.6466 - val_loss: 0.6488
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6934 - loss: 0.5848 - val_accuracy: 0.6663 - val_loss: 0.6185
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 927us/step - accuracy: 0.7203 - loss: 0.5535 - val_accuracy: 0.7335 - val_loss: 0.5502
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 925us/step - accuracy: 0.7424 - loss: 0.5253 - val_accuracy: 0.7710 - val_loss: 0.4822
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 905us/step - accuracy: 0.7910 - loss: 0.4598 - val_accuracy: 0.8134 - val_loss: 0.4306
Epoch 6/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 965us/step - accuracy: 0.8236 - loss: 0.4216 - val_accuracy: 0.8302 - val_loss: 0.3944
Epoch 7/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8425 - loss: 0.3678 - val_accuracy: 0.8460 - val_loss: 0.3669
Epoch 8/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 907us/step - accuracy: 0.8461 - loss: 0.3557 - val_accuracy: 0.8

/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5758 - loss: 0.6821 - val_accuracy: 0.6328 - val_loss: 0.6773
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 931us/step - accuracy: 0.6804 - loss: 0.5976 - val_accuracy: 0.6407 - val_loss: 0.6896
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 924us/step - accuracy: 0.7074 - loss: 0.5694 - val_accuracy: 0.7769 - val_loss: 0.4881
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 916us/step - accuracy: 0.7564 - loss: 0.4945 - val_accuracy: 0.8105 - val_loss: 0.4296
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 922us/step - accuracy: 0.8068 - loss: 0.4339 - val_accuracy: 0.8470 - val_loss: 0.3844
Epoch 6/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 977us/step - accuracy: 0.8378 - loss: 0.3956 - val_accuracy: 0.8401 - val_loss: 0.3651
Epoch 7/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 992us/step - accuracy: 0.8476 - loss: 0.3640 - val_accuracy: 0.8618 - val_loss: 0.3351
Epoch 8/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8578 - loss: 0.3415 - val_accuracy: 0

/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5538 - loss: 0.7082 - val_accuracy: 0.7177 - val_loss: 0.5778
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 971us/step - accuracy: 0.6786 - loss: 0.5952 - val_accuracy: 0.7157 - val_loss: 0.5383
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 998us/step - accuracy: 0.7215 - loss: 0.5449 - val_accuracy: 0.7512 - val_loss: 0.5427
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7657 - loss: 0.4899 - val_accuracy: 0.8026 - val_loss: 0.4657
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 981us/step - accuracy: 0.8117 - loss: 0.4346 - val_accuracy: 0.8213 - val_loss: 0.3959
Epoch 6/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 986us/step - accuracy: 0.8364 - loss: 0.3938 - val_accuracy: 0.8638 - val_loss: 0.3538
Epoch 7/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8537 - loss: 0.3541 - val_accuracy: 0.8628 - val_loss: 0.3400
Epoch 8/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8677 - loss: 0.3269 - val_accuracy: 0.830

In [19]:
model = 'CNN'

for stock in stocks:
    cnn_accs, cnn_pres, cnn_recs, cnn_f1s = [], [], [], []
    X_test, y_test = dataset_loading(features, target, f'./{stock}_test_data.csv',)
    X_train_dl, X_test_dl, y_train_dl, y_test_dl = process_dl_dataset(X_train, X_test, y_train, y_test, target)
    
    for i in range(N):
        model_cnn = Sequential()
        model_cnn.add(Conv1D(64, 3, activation='sigmoid', input_shape=(X_train_dl.shape[1], X_train_dl.shape[2]))) # 添加 Conv1D 層，對時間序列進行卷積操作
        model_cnn.add(MaxPooling1D(2))  # 最大池化層
        model_cnn.add(Flatten()) # 展平層，將多維數據轉換為一維
        # model_cnn.add(Dense(64, activation='sigmoid'))
        model_cnn.add(Dense(1, activation='sigmoid'))  # 二元分類使用 sigmoid 激活函數
        model_cnn.compile(optimizer=Adam(learning_rate=9.65E-03), loss='binary_crossentropy', metrics=['accuracy'])
        model_cnn.fit(X_train_dl, y_train_dl, epochs=10, batch_size=32, validation_data=(X_test_dl, y_test_dl))
        # loss, accuracy = model_cnn.evaluate(X_test_dl, y_test_dl)

        y_pred = model_cnn.predict(X_test_dl)
        y_pred = (y_pred > 0.5).astype(int)  # 將概率轉換為 0 或 1
        accuracy = accuracy_score(y_test_dl, y_pred)
        precision = precision_score(y_test_dl, y_pred, average='macro')
        recall = recall_score(y_test_dl, y_pred, average='macro')
        f1 = f1_score(y_test_dl, y_pred, average='macro')
        cnn_accs.append(accuracy)
        cnn_pres.append(precision)
        cnn_recs.append(recall)
        cnn_f1s.append(f1)

    print(stock)
    print(f'CNN Accuracy: {np.mean(cnn_accs) * 100:.2f}%')
    print(f'CNN Precision: {np.mean(cnn_pres) * 100:.2f}%')
    print(f'CNN Recall: {np.mean(cnn_recs) * 100:.2f}%')
    print(f'CNN F1 Score: {np.mean(cnn_f1s) * 100 :.2f}')

Epoch 1/10


/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5952 - loss: 0.6651 - val_accuracy: 0.7044 - val_loss: 0.5999
Epoch 2/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 904us/step - accuracy: 0.7010 - loss: 0.5758 - val_accuracy: 0.7017 - val_loss: 0.5799
Epoch 3/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 925us/step - accuracy: 0.7286 - loss: 0.5300 - val_accuracy: 0.7445 - val_loss: 0.5774
Epoch 4/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7882 - loss: 0.4531 - val_accuracy: 0.7403 - val_loss: 0.5178
Epoch 5/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 824us/step - accuracy: 0.8309 - loss: 0.3917 - val_accuracy: 0.7528 - val_loss: 0.5130
Epoch 6/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 844us/step - accuracy: 0.8424 - loss: 0.3633 - val_accuracy: 0.7569 - val_loss: 0.4933
Epoch 7/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 841us/step - accuracy: 0.8481 - loss: 0.3469 - val_accuracy: 0.7514 - val_loss: 0.4984
Epoch 8/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8587 - loss: 0.3288 - val_accuracy: 0.7

/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6110 - loss: 0.6556 - val_accuracy: 0.6906 - val_loss: 0.6056
Epoch 2/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7037 - loss: 0.5777 - val_accuracy: 0.6906 - val_loss: 0.5683
Epoch 3/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 812us/step - accuracy: 0.7661 - loss: 0.4991 - val_accuracy: 0.7182 - val_loss: 0.5424
Epoch 4/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 819us/step - accuracy: 0.8106 - loss: 0.4296 - val_accuracy: 0.7127 - val_loss: 0.5666
Epoch 5/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 854us/step - accuracy: 0.8266 - loss: 0.3895 - val_accuracy: 0.7362 - val_loss: 0.5145
Epoch 6/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 904us/step - accuracy: 0.8479 - loss: 0.3514 - val_accuracy: 0.7693 - val_loss: 0.4891
Epoch 7/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 840us/step - accuracy: 0.8602 - loss: 0.3280 - val_accuracy: 0.7680 - val_loss: 0.4672
Epoch 8/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step - accuracy: 0.8654 - loss: 0.3186 - val_accuracy: 0

/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Applications/anaconda3/envs/brbm/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5562 - loss: 0.7004 - val_accuracy: 0.6671 - val_loss: 0.5886
Epoch 2/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 974us/step - accuracy: 0.7072 - loss: 0.5587 - val_accuracy: 0.6865 - val_loss: 0.5759
Epoch 3/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 974us/step - accuracy: 0.7337 - loss: 0.5277 - val_accuracy: 0.7624 - val_loss: 0.5167
Epoch 4/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7873 - loss: 0.4676 - val_accuracy: 0.7431 - val_loss: 0.5232
Epoch 5/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 993us/step - accuracy: 0.8435 - loss: 0.3945 - val_accuracy: 0.7707 - val_loss: 0.4600
Epoch 6/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 936us/step - accuracy: 0.8524 - loss: 0.3610 - val_accuracy: 0.7970 - val_loss: 0.4271
Epoch 7/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 976us/step - accuracy: 0.8481 - loss: 0.3430 - val_accuracy: 0.7956 - val_loss: 0.4121
Epoch 8/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8619 - loss: 0.3193 - val_accuracy: 0.7